In [1]:
!pip install pgmpy

     |████████████████████████████████| 348kB 2.7MB/s 


In [0]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np

# Setting up your model

### First, set the structure

In [0]:
bayesNet = BayesianModel()
bayesNet.add_node("B")
bayesNet.add_node("A")
bayesNet.add_node("R")

bayesNet.add_node("S")

bayesNet.add_edge("B", "A")
bayesNet.add_edge("B", "R")
bayesNet.add_edge("A", "S")

bayesNet.add_edge("R", "S")

### Then set up the relationships

In [0]:
cpd_B = TabularCPD('B', 2, values=[[.50], [.50]])
cpd_R = TabularCPD('R', 2, values=[[.2,.7], [.8,.3]],evidence=['B'], evidence_card=[2])
cpd_A = TabularCPD('A', 2, values=[[.5,.9], [.5,.1]],evidence=['B'], evidence_card=[2])
cpd_S = TabularCPD('S', 2, values=[[0, .4, .2, 1], [1, .6, .8, 0]],
                   evidence=['R', 'A'], evidence_card=[2, 2])

### Add the relationships to your models

In [0]:
bayesNet.add_cpds(cpd_B, cpd_R, cpd_A, cpd_S )

### Examine your graph

In [8]:
bayesNet.check_model()
print("Model is correct.")

Model is correct.


# Making inferences

In [0]:
solver = VariableElimination(bayesNet)

In [13]:
result = solver.query(variables=['S'], evidence={'B': 1})
print("S| B", result)


Eliminating: A: 100%|██████████| 2/2 [00:00<00:00, 259.39it/s]

S| B +------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.1120 |
+------+----------+
| S(1) |   0.8880 |
+------+----------+


In [15]:
result = solver.query(variables=['R'], evidence={'S': 1})
print("R| S", result)

Eliminating: B: 100%|██████████| 2/2 [00:00<00:00, 393.48it/s]

R| S +------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.6082 |
+------+----------+
| R(1) |   0.3918 |
+------+----------+


In [16]:
result = solver.query(variables=['A'], evidence={'S': 1})
print("A| S", result)

Eliminating: B: 100%|██████████| 2/2 [00:00<00:00, 400.60it/s]

A| S +------+----------+
| A    |   phi(A) |
+======+==========+
| A(0) |   0.9254 |
+------+----------+
| A(1) |   0.0746 |
+------+----------+


### Get all independencies

In [17]:
bayesNet.get_independencies()

(B _|_ S | R, A)
(A _|_ R | B)
(R _|_ A | B)
(S _|_ B | R, A)